In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import box, LineString
import os
from matplotlib import pyplot
from pyproj import Transformer

In [4]:
basedir = '/gpfs/space/home/etais/hpc_nikolaykozlovskiy/transformers_project/Traffic4cast/data_traffic'
road_graph_folder = 'road_graph'
spatial_data_folder = 'spatial_data'
preprocess_stage = 'population'
buffer_m = 50
city = 'london'
crs_reprj = {'london': 27700}
city_epsg = crs_reprj[city]

In [5]:
road_graph_nodes = pd.read_parquet(f'{basedir}/{road_graph_folder}/{city}/road_graph_nodes.parquet')
road_graph_edges = pd.read_parquet(f'{basedir}/{road_graph_folder}/{city}/road_graph_edges.parquet')

In [6]:
road_graph_edges = road_graph_edges.merge(road_graph_nodes[['node_id', 'x', 'y']], 
                                          left_on = 'u', 
                                          right_on = 'node_id'
                                         )
road_graph_edges = road_graph_edges.merge(road_graph_nodes[['node_id', 'x', 'y']], 
                                          left_on = 'v', 
                                          right_on = 'node_id', 
                                          suffixes = ('_start', '_end')
                                         )

In [7]:
road_graph_edges['geometry'] = road_graph_edges.apply(lambda row:
    LineString([[row['x_start'], row['y_start']],[row['x_end'], row['y_end']]]), axis=1
)

In [8]:
gdf_road_graph_edges = gpd.GeoDataFrame(
    road_graph_edges,
    crs = 4326,
    geometry = 'geometry'
)

In [9]:
gdf_road_graph_edges = gdf_road_graph_edges.to_crs(city_epsg)
gdf_road_graph_edges['geometry'] = gdf_road_graph_edges['geometry'].buffer(buffer_m)

In [19]:
transformer = Transformer.from_crs(city_epsg, 4326, always_xy=True)
bounds_4326 = transformer.transform_bounds(*gdf_road_graph_edges.total_bounds)

In [18]:
population = pd.read_csv(f'{basedir}/{spatial_data_folder}/{preprocess_stage}/{city}/raw/population.csv')

In [20]:
population = population[population['Lat'].between(bounds_4326[1], bounds_4326[3])&
                        population['Lon'].between(bounds_4326[0], bounds_4326[2])
                        ].reset_index(drop=True)

In [26]:
gdf_population = gpd.GeoDataFrame(population, 
                                  geometry = gpd.points_from_xy(population['Lon'], population['Lat']), 
                                  crs = 4326
                                  ).to_crs(city_epsg)

In [27]:
sjoin = gpd.sjoin(gdf_road_graph_edges, gdf_population, predicate='contains')

In [30]:
mean_population = sjoin.groupby(['u', 'v'])['Population'].mean().reset_index()

In [32]:
road_graph_edges = road_graph_edges.merge(mean_population, on = ['u', 'v'], how='left')

In [34]:
road_graph_edges['Population'].fillna(0, inplace=True)